In [1]:
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.diagnostic import het_breuschpagan
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


## First, load the dataset from the weatherinszeged table from Thinkful's database.
Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?


In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   date                 96453 non-null  datetime64[ns, UTC]
 1   summary              96453 non-null  object             
 2   preciptype           96453 non-null  object             
 3   temperature          96453 non-null  float64            
 4   apparenttemperature  96453 non-null  float64            
 5   humidity             96453 non-null  float64            
 6   windspeed            96453 non-null  float64            
 7   windbearing          96453 non-null  float64            
 8   visibility           96453 non-null  float64            
 9   loudcover            96453 non-null  float64            
 10  pressure             96453 non-null  float64            
 11  dailysummary         96453 non-null  object             
 12  tempdiff          

In [3]:
df['tempdiff']=df.apparenttemperature-df.temperature
X=df[['humidity','windspeed']]
y=df['tempdiff']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train_const).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               tempdiff   R-squared:                       0.289
Model:                            OLS   Adj. R-squared:                  0.289
Method:                 Least Squares   F-statistic:                 1.567e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:54:06   Log-Likelihood:            -1.3620e+05
No. Observations:               77162   AIC:                         2.724e+05
Df Residuals:                   77159   BIC:                         2.724e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4379      0.023    104.020      0.000       2.392       2.484
humidity      -3.0282      0.027   -113.311      0.000      -3.081      -2.976
windspeed     -0.1190      0.001   -157.870      0.000      -0.120      -0.117
==============================================================================
Omnibus:                     3096.421   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3635.635
Skew:                          -0.471   Prob(JB):                         0.00
Kurtosis:                       3.492   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R2 and aR2=.289, which seems quite low. This means that there are probably siginifcant factors that are unobserved that are determining the variation that we're seeing in the dependent variable.

### Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one?


In [4]:
X['wind_humidity']=X.humidity*X.windspeed
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train_const).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               tempdiff   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.342
Method:                 Least Squares   F-statistic:                 1.335e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        21:56:26   Log-Likelihood:            -1.3322e+05
No. Observations:               77162   AIC:                         2.664e+05
Df Residuals:                   77158   BIC:                         2.665e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.1056      0.037      2.837      0.005       0.033       0.179
humidity          0.1488      0.048      3.111      0.002       0.055       0.243
windspeed         0.0889      0.003     32.463      0.000       0.083       0.094
wind_humidity    -0.2943      0.004    -78.736      0.000      -0.302      -0.287
==============================================================================
Omnibus:                     3874.274   Durbin-Watson:                   1.993
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7621.547
Skew:                          -0.369   Prob(JB):                         0.00
Kurtosis:                       4.351   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

R2 and aR2=.342, which is better, but probably still not great. It means that approximately 34% of the variance can be explained by the model.

### Add visibility as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the improvement in the adjusted R-squared. Which one is more useful?


In [7]:
X=df[['humidity','windspeed','visibility']]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

model = sm.OLS(y_train, X_train_const).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               tempdiff   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                 1.125e+04
Date:                Tue, 01 Sep 2020   Prob (F-statistic):               0.00
Time:                        22:00:17   Log-Likelihood:            -1.3535e+05
No. Observations:               77162   AIC:                         2.707e+05
Df Residuals:                   77158   BIC:                         2.707e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5833      0.031     51.024      0.000       1.522       1.644
humidity      -2.6112      0.028    -92.321      0.000      -2.667      -2.556
windspeed     -0.1196      0.001   -160.378      0.000      -0.121      -0.118
visibility     0.0535      0.001     41.434      0.000       0.051       0.056
==============================================================================
Omnibus:                     3016.971   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3616.395
Skew:                          -0.453   Prob(JB):                         0.00
Kurtosis:                       3.552   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R2 and aR2 actually decreased to .304. The second model is probably best, if we're going by R2 scores. The interaction term is therefore a more useful addition to the model.

### Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.


The second model also has the best (lowest) BIC score. It's also the only one whose direction of the parameters make sense to me.